# EXTERNAL MAPS FOR SYSTEMATICS

 Pixelweight format with short header
 Read already created maps and put them into pixweight format

In [1]:
import numpy as np
import healpy as hp
import requests
import fitsio
from fitsio import FITS,FITSHDR
import matplotlib as plt
from temdriver import rotate_map

### Path and names  

In [2]:
# Read already downloaded external maps
pathin = '/global/u1/m/manera/myKP3/kp3sys-inputs/'
pathout = '/global/u1/m/manera/myKP3/kp3sys-outputs/'

In [3]:
# check file names exist 
#!ls -c1 /global/u1/m/manera/myKP3/kp3sys-outputs
#!ls -c1 /global/u1/m/manera/myKP3/kp3sys-inputs

In [4]:
file_lensmap = pathout+'lensmap_nside256.fits'
file_lensmask = pathout+'lensmask_nside256.fits'

file_halmap = pathout+'Halpha_map_nside256.fits'
file_halmask = pathout+'Halpha_mask_nside256.fits'

file_ebvmap = pathout+'EBVext_map_nside256.fits'
file_ebvmask = pathout+'EBVext_MASK_map_nside256.fits'

file_mean_f6  = pathout+'EBVrecon_Mean_fw6p1_nside256.fits'
file_mean_f15 = pathout+'EBVrecon_Mean_fw15_nside256.fits'

file_var_f6   = pathout+'EBVrecon_Var_fw6p1_nside256.fits'
file_var_f15  = pathout+'EBVrecon_Var_fw15_nside256.fits'

file_mask_f6  = pathout+'EBVrecon_Mask_fw6p1_nside256.fits'
file_mask_f15 = pathout+'EBVrecon_Mask_fw15_nside256.fits'

file_calib_g  = pathout+'calib_ps1-g_nside256.fits'
file_calib_r  = pathout+'calib_ps1-r_nside256.fits'
file_calib_z  = pathout+'calib_ps1-z_nside256.fits'

file_calib_mask_g  = pathout+'calib_mask-g_nside256.fits'
file_calib_mask_r  = pathout+'calib_mask-r_nside256.fits'
file_calib_mask_z  = pathout+'calib_mask-z_nside256.fits'

#### READING MAPS AND CONVERTING FROM RING TO NEST 

In [5]:
def get_file_and_reorder(filename):
    aux = hp.read_map(filename,nest=False) 
    return hp.reorder(aux, inp='RING', out= 'NEST', r2n = True) 

In [ ]:
# DO NOT USE in new version
lensmap = hp.read_map(file_lensmap,nest=False) 
lensmap = hp.reorder(lensmap, inp='RING', out= 'NEST', r2n = True) 
lensmask = hp.read_map(file_lensmask,nest=False) 
lensmask = hp.reorder(lensmask, inp='RING', out= 'NEST', r2n = True) 

halmap = hp.read_map(file_halmap,nest=False) 
halmap = hp.reorder(halmap, inp='RING', out= 'NEST', r2n = True) 
halmask = hp.read_map(file_halmask,nest=False) 
halmask = hp.reorder(halmask, inp='RING', out= 'NEST', r2n = True)

ebvmap = hp.read_map(file_ebvmap,nest=False) 
ebvmap = hp.reorder(ebvmap, inp='RING', out= 'NEST', r2n = True) 
ebvmask = hp.read_map(file_ebvmask,nest=False) 
ebvmap = hp.reorder(ebvmask, inp='RING', out= 'NEST', r2n = True) 

In [6]:
######
hdict = [{'name':'HPXNSIDE','value' :256 , 'comment':'healpix nside'},
         {'name': 'HPXNEST','value' :True, 'comment':'healpix ordering'}]

In [41]:
######
#hdict = [{'name':'ORDERING','value':'NESTED', 'comment':'healpix order'},
#         {'name':'EFD1','value':'KAPPAPLANK','comment':'kappa values from Planck lensing'},
#         {'name':'EFD2','value':'KAPPAPLANKMASK','comment':'kappa mask values from Planck lensing'},
#         {'name':'EFD3','value':'HALPHA', 'comment':'Halpha fwhm06'},
#         {'name':'EFD4','value':'HALPHAMASK', 'comment': 'Halpha fwhm06 mask'},
#         {'name':'EFD5','value':'EBVext', 'comment': 'EBV from public map'},
#         {'name':'EFD6','value':'EBVextMASK', 'comment':'EBV mask from public map'}]

In [7]:
nside = 256
npix = hp.nside2npix(nside)
nrows = npix 
data = np.zeros(nrows, dtype=[('KAPPAPLANK','f8'),('KAPPAPLANKMASK','f8'),('HALPHA','f8'),
                              ('HALPHAMASK','f8'),('EBVext','f8'),('EBVextMASK','f8'),
                              ('CALIBG','f8'),('CALIBR','f8'),('CALIBZ','f8'),
                              ('CALIBMASKG','f8'),('CALIBMASKR','f8'),('CALIBMASKZ','f8'),
                              ('EBVreconMEANF6','f8'),('EBVreconMEANF15','f8'),
                              ('EBVreconVARF6','f8'),('EBVreconVARF15','f8'),
                              ('EBVreconMASKF6','f8'),('EBVreconMASKF15','f8'),
                             ])
data['KAPPAPLANK'] = get_file_and_reorder( file_lensmap  )
data['KAPPAPLANKMASK'] =  get_file_and_reorder( file_lensmask  )
data['HALPHA']     =  get_file_and_reorder( file_halmap  )
data['HALPHAMASK'] =  get_file_and_reorder( file_halmask )
data['EBVext']     =  get_file_and_reorder( file_ebvmap )
data['EBVextMASK'] =  get_file_and_reorder( file_ebvmask )

data['EBVreconMEANF6'] =  get_file_and_reorder( file_mean_f6   )
data['EBVreconMEANF15'] =  get_file_and_reorder(file_mean_f15 )
data['EBVreconVARF6'] =  get_file_and_reorder( file_var_f6 )
data['EBVreconVARF15'] =  get_file_and_reorder(file_var_f15  )
data['EBVreconMASKF6'] =  get_file_and_reorder( file_mask_f6  )
data['EBVreconMASKF15'] =  get_file_and_reorder( file_mask_f15)


data['CALIBG'] =  get_file_and_reorder( file_calib_g )
data['CALIBR'] =  get_file_and_reorder( file_calib_r )
data['CALIBZ'] =  get_file_and_reorder( file_calib_z )
data['CALIBMASKG'] =  get_file_and_reorder( file_calib_mask_g  )
data['CALIBMASKR'] =  get_file_and_reorder( file_calib_mask_r  )
data['CALIBMASKZ'] =  get_file_and_reorder( file_calib_mask_z  )

/global/common/software/desi/cori/desiconda/20200801-1.4.0-spec/conda/lib/python3.8/site-packages/healpy/fitsfunc.py:368: UserWarning: If you are not specifying the input dtype and using the default np.float64 dtype of read_map(), please consider that it will change in a future version to None as to keep the same dtype of the input file: please explicitly set the dtype if it is important to you.
  warnings.warn(
/global/common/software/desi/cori/desiconda/20200801-1.4.0-spec/conda/lib/python3.8/site-packages/healpy/fitsfunc.py:391: UserWarning: NSIDE = 256
  warnings.warn("NSIDE = {0:d}".format(nside))
/global/common/software/desi/cori/desiconda/20200801-1.4.0-spec/conda/lib/python3.8/site-packages/healpy/fitsfunc.py:400: UserWarning: ORDERING = RING in fits file
  warnings.warn("ORDERING = {0:s} in fits file".format(ordering))
/global/common/software/desi/cori/desiconda/20200801-1.4.0-spec/conda/lib/python3.8/site-packages/healpy/fitsfunc.py:428: UserWarning: INDXSCHM = IMPLICIT
  war

In [58]:
#### OLD NOT USED 
nside = 256
npix = hp.nside2npix(nside)
nrows = npix 
data = np.zeros(nrows, dtype=[('KAPPAPLANK','f8'),('KAPPAPLANKMASK','f8'),('HALPHA','f8'),
                              ('HALPHAMASK','f8'),('EBVext','f8'),('EBVextMASK','f8')])
data['KAPPAPLANK'] = lensmap
data['KAPPAPLANKMASK'] = lensmask
data['HALPHA'] = halmap
data['HALPHAMASK'] = halmask
data['EBVext'] = ebvmap
data['EBVextMASK'] = ebvmask


In [8]:
# CAREFUL, IF FILE EXISTS, the header will not be changed, but no warning is given. 
file_pixweight = 'pixweight_external.fits'
filename = pathout+file_pixweight
fits = FITS(filename,'rw')
fits.write(data, header=hdict)
hdr=FITSHDR(hdict)
fits.write(data, header=hdr)
fits.close()

In [37]:
#--------------------------------

Test by Reading 

In [9]:
data,h = fitsio.read(filename, header=True)

In [10]:
h


XTENSION= 'BINTABLE'           / binary table extension
BITPIX  =                    8 / 8-bit bytes
NAXIS   =                    2 / 2-dimensional binary table
NAXIS1  =                  144 / width of table in bytes
NAXIS2  =               786432 / number of rows in table
PCOUNT  =                    0 / size of special data area
GCOUNT  =                    1 / one data group (required keyword)
TFIELDS =                   18 / number of fields in each row
TTYPE1  = 'KAPPAPLANK'         / label for field   1
TFORM1  = 'D'                  / data format of field: 8-byte DOUBLE
TTYPE2  = 'KAPPAPLANKMASK'     / label for field   2
TFORM2  = 'D'                  / data format of field: 8-byte DOUBLE
TTYPE3  = 'HALPHA'             / label for field   3
TFORM3  = 'D'                  / data format of field: 8-byte DOUBLE
TTYPE4  = 'HALPHAMASK'         / label for field   4
TFORM4  = 'D'                  / data format of field: 8-byte DOUBLE
TTYPE5  = 'EBVext'             / label for field